In [2]:
import plotly.graph_objects as go
import numpy as np
import os

# モデルのパスや埋め込み範囲などのパラメータの設定
model_path = '/home/lab/eight/KGE-HAKE/models/HAKE_wn18rr_0'
embedding_range = 0.01
head_id = 40630 #22311	mammal.n.01 16430	food.n.01
tail_id = 16430	#27	dog.n.01 40630	pizza.n.01
num_bins = 100  # ヒストグラムのビンの数

# エンティティ名の辞書読み込み
entities_dict_file = '/home/lab/eight/KGE-HAKE/data/wn18rr_text/entities.dict'
entities_dict = {}
with open(entities_dict_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        entities_dict[int(parts[0])] = parts[1]
        
# エンティティの埋め込みをロード
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))

In [8]:

# headとtailの各座標を計算
phase, mod = np.log(np.split(entity_embedding, 2))

# 対数を取得
log_distances = np.log(distances + 1e-10)  # 0にならないように微小な値を加えています

# ヒストグラムの作成
hist = np.histogram(mod, bins=num_bins, range=(np.min(log_distances), np.max(mod)))

# Plotlyを使用してヒストグラムをプロット
fig = go.Figure()
fig.add_trace(go.Bar(x=hist[1][:-1], y=hist[0], name="Log Distance", opacity=0.8))

# レイアウトの設定
fig.update_layout(
    title='Histogram of Logarithm of Entity Embedding Distances',
    xaxis=dict(title='Log Distance'),
    yaxis=dict(title='Count'),
)

# プロットの表示
fig.show()

ValueError: array split does not result in an equal division

In [4]:
# エンティティの埋め込みをロード
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))

# headとtailの各座標を計算
phase_head, mod_head = np.split(entity_embedding[head_id], 2)
phase_tail, mod_tail = np.split(entity_embedding[tail_id], 2)

# ログ変換と極座標への変換
mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

# 極座標から直交座標への変換
x_head, y_head = mod_head * np.cos(phase_head), mod_head * np.sin(phase_head)
x_tail, y_tail = mod_tail * np.cos(phase_tail), mod_tail * np.sin(phase_tail)

# 距離の計算
distance_head = np.sqrt(x_head**2 + y_head**2)
distance_tail = np.sqrt(x_tail**2 + y_tail**2)

# ヒストグラムの作成
hist_head = np.histogram(distance_head, bins=num_bins, range=(0, np.max(distance_head)))
hist_tail = np.histogram(distance_tail, bins=num_bins, range=(0, np.max(distance_tail)))

# Plotlyを使用してヒストグラムをプロット
fig = go.Figure()
fig.add_trace(go.Bar(x=hist_head[1][:-1], y=hist_head[0], name="head "+entities_dict[head_id], opacity=0.8))

# Tail Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_tail[1][:-1], y=hist_tail[0], name="tail "+entities_dict[tail_id], opacity=0.8))


# レイアウトの設定
fig.update_layout(
    title='Histograms of the modulus of entity embeddings',
    xaxis=dict(title='原点からの距離'),
    yaxis=dict(title='カウント'),
    barmode='overlay',  # HeadとTailのヒストグラムをオーバーレイ
)

# プロットの表示
fig.show()


In [5]:
# 平均距離の計算
mean_distance_head = np.mean(np.sqrt(x_head**2 + y_head**2))
mean_distance_tail = np.mean(np.sqrt(x_tail**2 + y_tail**2))

# 結果の出力
print(f"平均距離（Head）: {mean_distance_head}")
print(f"平均距離（Tail）: {mean_distance_tail}")

平均距離（Head）: 1.8628108501434326
平均距離（Tail）: 6.607659339904785


In [6]:
# カウントの合計数を計算
head_count = np.sum(hist_head[0])
tail_count = np.sum(hist_tail[0])

# 結果の出力
print(f"表示Entity（Head）: {head_count}")
print(f"表示Entity（Tail）: {tail_count}")

表示Entity（Head）: 500
表示Entity（Tail）: 500
